# ENIGMA Wiki-Bot

The ENIGMA Wiki-Bot can be used to retrieve statistics about the ENIGMA Wiki (http://organicdatacuration.org/enigma_new/index.php/Main_Page) and write them as a Digest on the Main Page. 

To access your own bot, get the bot login credentials from http://organicdatacuration.org/enigma_new/index.php/Special:BotPasswords and include the username and password in the <b>Logging In</b> section below. Then run the rest of the cells to add the updated digest to the wiki!

NOTE: Once you run this notebook, the previous contents of the Digest will be removed and updated with today's statistics. Currently, the Digest corresponds to Section 16 in the Main Page. If any sections have been added (or deleted) after July 30, 2020, please update the "section" value in the last cell. Thanks!

## Logging In

In [1]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup as bs

In [2]:
S = requests.Session()

URL = "http://organicdatacuration.org/enigma_new/api.php"

# Retrieve login token
PARAMS_0 = {
    'action':"query",
    'meta':"tokens",
    'type':"login",
    'format':"json"
}


DATA = S.get(url=URL, params=PARAMS_0).json()
LOGIN_TOKEN = DATA['query']['tokens']['logintoken']

print("Login Token: ",LOGIN_TOKEN)

# Go to http://organicdatacuration.org/enigma_new/index.php/Special:BotPasswords for lgname & lgpassword, and include them below

PARAMS_1 = {
    'action':"login",
    'lgname':"",
    'lgpassword':"",
    'lgtoken':LOGIN_TOKEN,
    'format':"json"
}

DATA = S.post(URL, data=PARAMS_1).json()

print(DATA)

Login Token:  b9b22f4dcdb0aed69d7de3786beed7e25f23404c+\
{'login': {'result': 'Success', 'lguserid': 96, 'lgusername': 'Priya'}}


## Retrieve Info

In [3]:
def get_stats(query):
    response = S.get(query).json()
    result = response['query']['pages'].values()
    count = []
    [[count.append(j['categoryinfo']['size'])] for j in result]
    return count[0]

num_datasets = get_stats("http://organicdatacuration.org/enigma_new/api.php?action=query&prop=categoryinfo&titles=Category:Dataset_(E)&format=json")
num_organizations = get_stats("http://organicdatacuration.org/enigma_new/api.php?action=query&prop=categoryinfo&titles=Category:Organization_(E)&format=json")
num_persons = get_stats("http://organicdatacuration.org/enigma_new/api.php?action=query&prop=categoryinfo&titles=Category:Person_(E)&format=json")
num_cohorts = get_stats("http://organicdatacuration.org/enigma_new/api.php?action=query&prop=categoryinfo&titles=Category:Cohort_(E)&format=json")

statistics = {'datasets':num_datasets,
             'organizations':num_organizations,
             'persons':num_persons,
             'cohorts':num_cohorts}
print(statistics)

{'datasets': 87, 'organizations': 508, 'persons': 549, 'cohorts': 227}


## Writing to Wiki 

In [16]:
##Run this cell to create today's Digest!
dt = datetime.now()
timestamp = dt.strftime("%B %d, %Y %H:%M:%S")
##The following digest will be added to the Wiki
digest = "==Digest==\n\nAs of "+timestamp+", there are a total of "+str(statistics['datasets'])+" '''datasets''' in the ENIGMA Wiki which belong to "+str(statistics['cohorts'])+" '''cohorts'''. There are "+str(statistics['persons'])+" '''persons''' associated with "+str(statistics['organizations'])+" '''organizations.'''"
print(digest)

==Digest==

As of July 30, 2020 15:25:25, there are a total of 87 '''datasets''' in the ENIGMA Wiki which belong to 227 '''cohorts'''. There are 549 '''persons''' associated with 508 '''organizations.'''


In [17]:
##Execute this to add today's digest to the wiki
PARAMS_2 = {
    "action": "query",
    "meta": "tokens",
    "format": "json"
}

R = S.get(url=URL, params=PARAMS_2)
DATA = R.json()

CSRF_TOKEN = DATA['query']['tokens']['csrftoken']

# Step 4: POST request to edit a page

#If any sections have been added/deleted post 7/30/2020, modify the section number here.
PARAMS_EDIT = {
    "action": "edit",
    "title": "Main_Page",
    "section": "16",
    "format": "json",
    "text": digest,
    "token": CSRF_TOKEN,
}

R = S.post(URL, data=PARAMS_EDIT)
DATA = R.json()

print(DATA)

{'edit': {'result': 'Success', 'pageid': 1, 'title': 'Main Page', 'contentmodel': 'wikitext', 'oldrevid': 23473, 'newrevid': 23474, 'newtimestamp': '2020-07-30T22:13:13Z'}}
